In [1]:
from pathlib import Path
import re
from collections import Counter
import math

import pandas as pd
import matplotlib.pyplot as plt

# Define directories
DATA_DIR = Path("../../data") if Path(".").name != "applied-NLP-project" else Path("data")
RESULTS_DIR = Path("../../results") if Path(".").name != "applied-NLP-project" else Path("results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

BOOKS = {
    "War_and_Peace": DATA_DIR / "War_and_Peace.txt",
    "Anna_Karenina": DATA_DIR / "Anna_Karenina.txt",
}


ModuleNotFoundError: No module named 'pandas'